# Processing to have final data 
After annotate the meta data of census and have the samples matching

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

from PopSynthesis.Generator_data.generate_combine_census.utils import *

In [2]:
# Import the meta data
meta_df_hh = pd.read_csv("./processed_data/meta_data_hh_manual.csv")
meta_df_pp = pd.read_csv("./processed_data/meta_data_pp_manual.csv")

In [3]:
geo_lev = "POA"
data_raw = process_from_census_data(geo_lev=geo_lev)

In [4]:
persons_cols = [x for x in data_raw.columns if "Person_" in x]
hh_cols = [x for x in data_raw.columns if "Dwelling_" in x]

pp_marg_raw = data_raw[persons_cols]
hh_marg_raw = data_raw[hh_cols]

In [6]:
cols_have_geo_hh = [x for x in hh_marg_raw.columns if geo_lev in x]
assert len(cols_have_geo_hh) == 1
tot_hh_seri = hh_marg_raw[cols_have_geo_hh[0]]

cols_have_geo_pp = [x for x in pp_marg_raw.columns if geo_lev in x]
assert len(cols_have_geo_pp) == 1
tot_pp_seri = pp_marg_raw[cols_have_geo_pp[0]]

In [36]:
def process_data_all(type_att="Dwelling"):
    if type_att == "Dwelling":
        meta_data, raw_marg_data, tot_seri = meta_df_hh, hh_marg_raw, tot_hh_seri
    elif type_att == "Person":
        meta_data, raw_marg_data, tot_seri = meta_df_pp, pp_marg_raw, tot_pp_seri
    
    ls_df_to_concat = []
    for att_code in meta_data["att_code_census"].unique():
        dict_hold_val = {}

        sub_df = meta_data[meta_data["att_code_census"]==att_code]
        assert len(sub_df["att_description"].unique()) == 1
        assert len(sub_df["att_sample"].unique()) == 1
        
        name_full = type_att + "_" + att_code + " " + sub_df["att_description"].unique()[0]
        att_sample = sub_df["att_sample"].unique()[0]
        all_state_census = sub_df["state_census"].unique()
        all_cols_name = [f"{name_full}__{state_census}" for state_census in all_state_census]
        gb_df = sub_df.groupby("state_sample")["state_census"].apply(lambda x: list(x))

        # Process delete all and shift the value to others cols
        re_df = raw_marg_data.fillna(0)
        if "DELETE" in gb_df.index:
            sub_del_name = gb_df["DELETE"]
            cols_del = [f"{name_full}__{del_census}" for del_census in sub_del_name]
            ls_not_del = list(set(all_cols_name) - set(cols_del))
            to_plus_seri = raw_marg_data[cols_del].sum(axis=1) / len(ls_not_del)
            re_df = raw_marg_data[ls_not_del].add(to_plus_seri, axis="index")

        # Process the break when we want the value to shared between specific cols
        for att_sample_state in gb_df.index:
            if "BREAK" in att_sample_state:
                ls_cols_to_take = att_sample_state.split(":")[1].split("|")
                hold_cols = []
                for x in ls_cols_to_take:
                     hold_cols += gb_df[x]
                col_del = f"{name_full}__{gb_df[att_sample_state][0]}"
                cols_to_update = [f"{name_full}__{col_take}" for col_take in hold_cols]
                sub_df_of_this = re_df[cols_to_update].add((re_df[col_del] / len(ls_cols_to_take)), axis="index")
                # Update the final df
                re_df[cols_to_update] = sub_df_of_this
                re_df = re_df.drop(columns=col_del)

        for att_sample_state in gb_df.index:
            if "BREAK" not in att_sample_state and "DELETE" != att_sample_state:
                ls_of_census = [f"{name_full}__{census_val}" for census_val in gb_df[att_sample_state]]
                final_seri_val = re_df[ls_of_census].sum(axis=1)
                dict_hold_val[(att_sample, att_sample_state)] = final_seri_val
        df_of_att = pd.DataFrame(dict_hold_val)
        # Process to intergalised the re_df
        after_inter = update_int_all(df_of_att, tot_seri)
        ls_df_to_concat.append(after_inter)

    final_df = pd.concat(ls_df_to_concat, axis=1)
    final_df[("zone_id", None)] = final_df.index
    final_df[("sample_geog", None)] = 2
    return final_df

In [37]:
final_df_census_hh = process_data_all(type_att="Dwelling")
final_df_census_pp = process_data_all(type_att="Person")

In [38]:
final_df_census_hh.to_csv("./hh_marginals_ipu.csv", index=False)
final_df_census_pp.to_csv("./person_marginals_ipu.csv", index=False)

## Process the sample data now

In [40]:
ori_sample_hh = pd.read_csv("./processed_data/ori_sample_hh.csv")
ori_sample_pp = pd.read_csv("./processed_data/ori_sample_pp.csv")

In [44]:
ipu_sample_hh = ori_sample_hh.drop(columns=["Spouse","Child","Others","Grandchild","_weight"]).rename(columns={"hhid": "serialno"})
ipu_sample_pp = ori_sample_pp.drop(columns=["persid", "_weight"]).rename(columns={"hhid": "serialno"})
ipu_sample_hh["sample_geog"] = 2
ipu_sample_pp["sample_geog"] = 2

In [45]:
ipu_sample_pp

,serialno,age,sex,relationship,persinc,nolicence,anywork,sample_geog
0,Y12H0000101,50-59,M,Main,$1000-1249 p.w.,Some Licence,Y,2
1,Y12H0000101,40-49,F,Spouse,$1-199 p.w.,Some Licence,N,2
2,Y12H0000101,10-19,F,Child,Zero Income,No Licence,N,2
3,Y12H0000101,0-9,F,Child,Zero Income,No Licence,N,2
4,Y12H0000102,50-59,F,Main,$1-199 p.w.,Some Licence,N,2
...,...,...,...,...,...,...,...,...
78973,Y20H4001028,60-69,F,Spouse,$400-599 p.w.,Some Licence,Y,2
78974,Y20H4001029,70-79,M,Main,$300-399 p.w.,Some Licence,N,2
78975,Y20H4001029,70-79,F,Spouse,$300-399 p.w.,Some Licence,N,2
78976,Y20H4001036,70-79,F,Main,$400-599 p.w.,Some Licence,N,2


In [46]:
ipu_sample_pp.to_csv("./pp_sample_ipu.csv", index=False)
ipu_sample_hh.to_csv("./hh_sample_ipu.csv", index=False)